In [1]:
import pandas as pd
import sqlite3

In [18]:
db_path = 'data/broadcast_licenses.db'
org_query = """
SELECT r.id, r.org_id, r.org_name, r.smi_name,
       g.id   AS grid_id, g.region_name_full, g.region_text, g.channel_num, g.freq, g.power, g.population
FROM records AS r
INNER JOIN broadcast_grid AS g
  ON r.id = g.record_id
WHERE r.status = 'действующая'
  AND r.licensed_activity = 'Радиовещание радиоканала'
  AND g.region_name_full != 'Российская Федерация';
"""

conn = sqlite3.connect(db_path)
df = pd.read_sql_query(org_query, conn)
conn.close()

df

,id,org_id,org_name,smi_name,grid_id,region_name_full,region_text,channel_num,freq,power,population
0,15046,871,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РАДИ...",Радиостанция ХИТ-ФМ (163); ХИТ-ФМ - ОРЕНБУРГ (5),160213,Оренбургская область,Оренбург г,,"103,0",2,"620,0"
1,15047,1590034,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТЕЛЕ...",Дорожное радио (168),160214,Краснодарский край,Геленджик г,,"102,9","0,1","98,2"
2,15050,450534,"Общество с ограниченной ответственностью ""Поля...",РАДИО-ЛЮБОВЬ (168),160217,Республика Саха (Якутия),Якутск г,,"90,9",1,"301,0"
3,15055,504789,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВАЛЕ...",Городской романс (12); РАДИО-ЛЮБОВЬ (156),160222,Ульяновская область,Ульяновск г,,"106,2",1,"920,0"
4,15059,464353,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИЖЕВ...",Радио Адам (168),160226,Удмуртская Республика,Ижевск г,,"104,5",1,"850,0"
...,...,...,...,...,...,...,...,...,...,...,...
11490,27366,1432922,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ГПМ ...",Лайк FM (Like FM) (168),235767,Свердловская область,Каменск-Уральский г,,"103,5","0,25","212,4"
11491,27366,1432922,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ГПМ ...",Лайк FM (Like FM) (168),235768,Удмуртская Республика,Ижевск г,,"88,0",1,"672,5"
11492,27366,1432922,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ГПМ ...",Лайк FM (Like FM) (168),235769,Хабаровский край,Комсомольск-на-Амуре г,,"99,5","0,45","301,7"
11493,27366,1432922,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ГПМ ...",Лайк FM (Like FM) (168),235770,Челябинская область,Магнитогорск г,,"101,4","0,45","444,5"


In [19]:
def parse_float(item):
    if type(item) == float:
        return item
    first_item = item.strip().split(' ')[0]
    first_item = first_item.replace(',', '.')
    try:
        out = float(first_item)
    except ValueError: 
        out = None
    return out

In [20]:
df['population'] = df['population'].apply(parse_float)
len(df), len(df['population'].notna())

(11495, 11495)

In [21]:
result = df.groupby(['id', 'org_id', 'org_name', 'smi_name'])['population'].sum().reset_index()
result.sort_values(ascending=False, by='population')

,id,org_id,org_name,smi_name,population
319,16869,755034,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯ...,Радио России (168),171848.817
317,16867,755034,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯ...,Вести ФМ (168),116922.800
318,16868,755034,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯ...,Маяк (168),95470.641
3097,26822,1425640,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КОМП...",Новое радио (168),77959.404
3221,27030,578090,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""РУССКОЕ РАДИО-ЕВРАЗИЯ""",Русское Радио (168),70786.428
...,...,...,...,...,...
2194,24818,376008,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ОРИО...",Вести ФМ (168); Маяк (168); Радио России (168),0.000
3322,27182,5193420,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЦТВ ...",Вести ФМ (168); Маяк (168); Радио России (168),0.000
1998,24067,794103,"НЕПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""НАЦИОНАЛЬНАЯ...",Studio 21 (168); Авторадио (168); Вести ФМ (16...,0.000
1785,23138,755034,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯ...,Соотечественники (3),0.000


In [22]:
result.population.sum()

np.float64(3599238.602)

In [23]:
result.head(10)

,id,org_id,org_name,smi_name,population
0,15046,871,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РАДИ...",Радиостанция ХИТ-ФМ (163); ХИТ-ФМ - ОРЕНБУРГ (5),620.000
1,15047,1590034,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТЕЛЕ...",Дорожное радио (168),98.200
2,15050,450534,"Общество с ограниченной ответственностью ""Поля...",РАДИО-ЛЮБОВЬ (168),301.000
3,15055,504789,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВАЛЕ...",Городской романс (12); РАДИО-ЛЮБОВЬ (156),920.000
4,15059,464353,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИЖЕВ...",Радио Адам (168),850.000
5,15068,1802201,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РАДИ...",РАДИО-ЛЮБОВЬ (168),6000.000
6,15082,949978,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""БАЛТ...",Радио-Дача (168),5188.000
7,15093,491031,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ДО М...",Ретро FM (Retro FM) (167); Ретро-Мажор (1),125.000
8,15104,513783,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""САМА...",DFM Самара (10); Ди FM (D FM) (158),1278.371
9,15106,1437948,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РАДИ...",Радио Шансон (148); Радио Шансон Екатеринбург ...,1330.500
